In [1]:
# Useful for debugging
%load_ext autoreload
%autoreload 2

# SLAC XSIF to Bmad conversion

This requires code from LCLS_LATTICE

In [2]:
# Patch in the slac2bmad package
import sys, os
pypath = os.environ['LCLS_LATTICE']+'/bmad/conversion/python'
sys.path.append(pypath)

In [3]:
from slac2bmad.xsif import prepare_xsif, remove_comment_blocks, replace_set, replace_set_commands, fix_matrix, expand_names, fix_names, unfold_comments, fold_comments
from slac2bmad.desplit import desplit_eles, desplit_ele
from slac2bmad.replace import replace_element, replace_eles
from slac2bmad.bmad import finalize_bmad

from glob import glob
import shutil

import subprocess
import json
import os

# Custom element replacements (in Bmad)

In [4]:
NEWELES = {}

NEWELES['um10466'] = """
!------- Laser Heater Undulator for Copper Linac -------
my_lh_und_k = 0
um10466: wiggler, 
        type = "laser_heater_undulator",
        L_period = 0.054, 
        n_period = 10, 
        b_max = my_lh_und_k * 2*pi*m_electron / (c_light * 0.054), 
        L = 10*0.054 ! Was: 0.506263, 
        ds_step = 0.054
        
um10466[L] = um10466[L]/2 ! Will be doubled in desplitting process. 
!---------------------------------
    """
        
NEWELES['dh02b'] = """
! Shorten so that lh_und has an integer number of poles
dh02b: drift, l = 0.0846296264 - ( 10*0.054 - 0.506263 ) /2
"""

NEWELES['dh03a'] = """
! Shorten so that lh_und has an integer number of poles
dh03a: drift, l = 0.0845477112 - ( 10*0.054 - 0.506263 ) /2
"""    
    
    
 

In [5]:
# Add these repalcements
F2_LINAC_REPLACEMENTS = json.load(open('good_facet2_linac_replacements.json'))
for name, replace in F2_LINAC_REPLACEMENTS.items():
    NEWELES[name.lower()+'_full'] = replace   

In [6]:
BENDS_TO_DESPLIT = [
'BCX11314',
'BCX11331',
'BCX11338',
'BCX11355',
'BX10751',
'BX10661',    
'BCX14720',
'BCX14796',
'BCX14808',
'BCX14883',
'BCX141720',
'BCX141796',
'BCX141808',
'BCX141883',
'BCX10451',
'BCX10461',
'BCX10475',
'BCX10481',  
'B1LE',
'B2LE',
'B3LE',
'B3RE',
'B2RE',
'B1RE',
'BLX57172',
'BLY57174',
'BKY170',
'BY57202',
'BX57205',
'BX57215',
'BX57225',
'BX57235',
'BX57245',
'BX57255',
'BX57265',
'BX57275'
    
]
def desplit_bend_line(name):
    return f'{name}_full: line = ({name})'

BEND_REPLACEMENTS = {}
for name in BENDS_TO_DESPLIT:
    BEND_REPLACEMENTS[name+'_full'] = desplit_bend_line(name)
NEWELES.update(BEND_REPLACEMENTS )

In [7]:
def all_replacements(master_file):
    dat = {}
    dat.update(NEWELES)
    #if master_file.startswith('F2_'):
    #    print('F2replacements')
    #    dat.update(F2_NEWELES)
    #    return dat
    #else:
    #    raise 
    return dat
#all_replacements('F2_')        

# Convert all

In [8]:
!mkdir temp

In [9]:
# Clean
!rm *xsif *bmad *digested*

rm: *xsif: No such file or directory
rm: *bmad: No such file or directory
rm: *digested*: No such file or directory


In [10]:
!pwd

/Users/chrisonian/Code/GitHub/facet2-lattice/bmad/conversion


In [11]:
!cp /Users/chrisonian/Code/GitHub/facet2-lattice/mad/*xsif .

In [12]:
XSIF_FILES=[f for f in os.listdir() if f.endswith('.xsif')]
for f in XSIF_FILES:
    prepare_xsif(f, save=False)

Preparing INJ.xsif
Preparing QDDSQ.xsif
Preparing FACET2p_DRTBC11.xsif
Preparing FACET2e_master.xsif
Preparing F2_POSI.xsif
Preparing BA.xsif
Preparing SCAV.xsif
Preparing QF2Q.xsif
Preparing FACET2p_DR.xsif
Preparing FACET2p_master.xsif
Preparing BC14.xsif
Preparing L2.xsif
Preparing LI20.xsif
Preparing L3.xsif
Preparing F2_ELEC.xsif
Preparing common.xsif
Preparing BD.xsif
Preparing L1.xsif
Preparing F2_S10AIP.xsif
Preparing F2_SCAV.xsif
Preparing DL10.xsif
Preparing QFCQ.xsif
Preparing BC20E.xsif
Preparing FACET2p_PRLTDR.xsif
Preparing BC11.xsif
Preparing BC20P.xsif


In [13]:
!mv *xsif temp/

In [14]:
F2_MASTERS = [f for f in os.listdir('../../mad') if f.startswith('F2_')]
F2_MASTERS

['F2_POSI.xsif', 'F2_ELEC.xsif', 'F2_S10AIP.xsif', 'F2_SCAV.xsif']

In [15]:
TEMPDIR = './temp/'
WORKDIR = './work/'

In [16]:
!mkdir {TEMPDIR}
!mkdir {WORKDIR}

mkdir: ./temp/: File exists


# Process 

In [17]:
DEST = os.path.expandvars('$FACET2_LATTICE/bmad/master/')
assert os.path.exists(DEST)

In [18]:
def process_master(master):
    
    print(f'Converting {master}')
    
    shutil.copytree(TEMPDIR, WORKDIR, dirs_exist_ok=True)
    
    # New method
    SCRIPT = f'cd work;python $ACC_ROOT_DIR/util_programs/mad_to_bmad/mad8_to_bmad.py --no_prepend_vars -f {master}'
    print(SCRIPT)
    res = subprocess.run(SCRIPT, shell=True, cwd=WORKDIR)
    
    assert res.returncode == 0
    
    BMAD_FILES=glob(WORKDIR+'/*bmad')

    REPLACEMENTS = all_replacements(master)
    #REPLACEMENTS={}

    for f in BMAD_FILES:
        finalize_bmad(f, replacements=REPLACEMENTS, verbose=False)   
    
    print(f'    Copying all to {DEST}')
    for f in BMAD_FILES:
        #print(f'copying {f} to {DEST}')
        shutil.copy(f, DEST)
    
    
process_master('F2_ELEC.xsif')
#process_master('INJ.xsif')

Converting F2_ELEC.xsif
cd work;python $ACC_ROOT_DIR/util_programs/mad_to_bmad/mad8_to_bmad.py --no_prepend_vars -f F2_ELEC.xsif
    Copying all to /Users/chrisonian/Code/GitHub/facet2-lattice//bmad/master/


In [19]:
for m in F2_MASTERS:
    process_master(m)

Converting F2_POSI.xsif
cd work;python $ACC_ROOT_DIR/util_programs/mad_to_bmad/mad8_to_bmad.py --no_prepend_vars -f F2_POSI.xsif
    Copying all to /Users/chrisonian/Code/GitHub/facet2-lattice//bmad/master/
Converting F2_ELEC.xsif
cd work;python $ACC_ROOT_DIR/util_programs/mad_to_bmad/mad8_to_bmad.py --no_prepend_vars -f F2_ELEC.xsif
    Copying all to /Users/chrisonian/Code/GitHub/facet2-lattice//bmad/master/
Converting F2_S10AIP.xsif
cd work;python $ACC_ROOT_DIR/util_programs/mad_to_bmad/mad8_to_bmad.py --no_prepend_vars -f F2_S10AIP.xsif
    Copying all to /Users/chrisonian/Code/GitHub/facet2-lattice//bmad/master/
Converting F2_SCAV.xsif
cd work;python $ACC_ROOT_DIR/util_programs/mad_to_bmad/mad8_to_bmad.py --no_prepend_vars -f F2_SCAV.xsif
    Copying all to /Users/chrisonian/Code/GitHub/facet2-lattice//bmad/master/


# Final cleanup

In [20]:
!rm -r {TEMPDIR}
!rm -r {WORKDIR}